<a href="https://colab.research.google.com/github/EdwynZN/TLGColab/blob/main/DS_C2_SC2_EdwinZambranoNemegyei.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import os

def sumVaccinesRow(row) -> pd.Series:
    return row['daily_vaccinations'] * len(row['vaccines'].split(','))

def vaccinesSpecificDate(dataframe: pd.DataFrame, date: pd.Timestamp) -> int:
    filter = dataframe[dataframe['date'] == date]
    return filter.apply(sumVaccinesRow, axis=1).sum()

def deltaTime(dataframe: pd.DataFrame, dateName: str) -> pd.Timedelta:
    firstDay = dataframe[dateName].min()
    lastDay = dataframe[dateName].max()
    return lastDay - firstDay

def dataFrameBeforeDate(dataframe: pd.DataFrame, dateName: str, date: pd.Timestamp) -> pd.DataFrame:
    return dataframe.loc[dataframe[dateName] < str(date)]

def dataFrameOfManufacturer(dataframe: pd.DataFrame, columnName: str, manufacturer: str) -> pd.DataFrame:
    return dataframe.loc[dataframe[columnName].str.contains(manufacturer, case=False)]

def excelWriter(path: str) -> pd.ExcelWriter:
    if os.path.exists(path):
        return pd.ExcelWriter(path, if_sheet_exists='replace', mode='a')
    else:
        return pd.ExcelWriter(path)

# A. Extraer información del csv
vaccines = pd.read_csv('country_vaccinations.csv', parse_dates=['date'])

# B. Estructura y tipo de datos
print('tipo de datos de vacunas:', vaccines.dtypes, sep='\n')

# C. Vacunas aplicadas por compañía
manufacturerDataFrame = vaccines.vaccines.str.get_dummies(', ').multiply(vaccines.daily_vaccinations, 0).sum()
print('Vacunas aplicadas por compañia:', manufacturerDataFrame, sep='\n')

# D. Vacunas aplicadas en todo el mundo
print('Vacunas aplicadas en todo el mundo: ', vaccines.apply(sumVaccinesRow, axis=1).sum())

# E. Promedio de vacunas por país
meanCountry = vaccines.loc[:, ['country', 'daily_vaccinations', 'vaccines']]
meanCountry['daily_vaccinations'] = meanCountry.apply(sumVaccinesRow, axis=1)
meanCountry = meanCountry[['country', 'daily_vaccinations']].groupby('country').mean()
print('Promedio de vacunados por país', meanCountry.head(8), sep='\n')

# F. Determinar la cantidad de vacunas aplicadas el día 29/01/2021 en todo el mundo.
vaccinesInADay = vaccinesSpecificDate(vaccines, pd.to_datetime('29/01/21', dayfirst=True))
print('Vacunas aplicadas en todo el mundo el día 29/01/2021: ', vaccinesInADay)

# G. Diferencias de aplicación entre las columnas daily_vaccinations y daily_vaccinations_raw.  
conDiferencias = vaccines.copy()
conDiferencias['diffDaily'] = conDiferencias['daily_vaccinations'].fillna(0) - conDiferencias['daily_vaccinations_raw'].fillna(0)
print('Diferencias entre las vacunas diarias y el dato en crudo', conDiferencias['diffDaily'], sep='\n')

# H. Obtener el período de tiempo entre el registro con fecha más reciente y el registro con fecha más antigua.
period = deltaTime(vaccines, 'date')
print('Periodo de tiempo en días: ', period.days)

# I. Cantidad de vacunas utilizadas cada día
conCantidad = vaccines.copy()
conCantidad['canVac'] = conDiferencias['daily_vaccinations'] * conDiferencias['vaccines'].str.split(',').str.len()
print('Vacunas aplicadas en todo el mundo: ', conCantidad[['country', 'date', 'canVac']].head(10), sep='\n')

# J. DataFrame con todos los registros que se hayan realizado antes del 20 de diciembre de 2020
antes20 = dataFrameBeforeDate(vaccines, 'date', pd.to_datetime('20/12/20', dayfirst=True))
print('Vacunas aplicadas antes del 20 de diciembre del 2020', antes20.tail(10), sep='\n')

# K. DataFrame con todos los registros donde se haya utilizado la vacuna Pfizer
pfizer = dataFrameOfManufacturer(vaccines, 'vaccines', 'Pfizer')
print('Registro de vacunas Pfizer', pfizer.tail(10), sep='\n')

# L. Guardar nuevos DataFrames.
fileName = 'sample_data/resultadosReto.xlsx'
with excelWriter(fileName) as writer:
    conDiferencias.to_excel(writer, sheet_name="conDiferencias")
    conCantidad.to_excel(writer, sheet_name="conCantidad")
    antes20.to_excel(writer, sheet_name="antes20")
    pfizer.to_excel(writer, sheet_name="pfizer")